# Deploy to an batch endpoint

Imagine a health clinic takes patient measurements all day, saving the details for each patient in a separate file. Then overnight, the diabetes prediction model can be used to process all of the day's patient data as a batch, generating predictions that will be waiting the following morning so that the clinic can follow up with patients who are predicted to be at risk of diabetes. With Azure Machine Learning, you can accomplish this by creating a batch endpoint; and that's what you'll implement in this exercise.

## Before you start

You'll need the latest version of the  **azureml-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.9.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: tqdm, marshmallow, azure-storage-file-share, typing-extensions, azure-storage-file-datalake, pyyaml, pyjwt, pydash, azure-common, opencensus-ext-azure, azure-mgmt-core, msrest, strictyaml, isodate, azure-storage-blob, colorama, azure-core, jsonschema
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [2]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential=DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential=InteractiveBrowserCredential()


In [3]:
ml_client=MLClient.from_config(credential=credential)

Found the config file in: /config.json


## Register the model

Batch deployments can only deploy models registered in the workspace. You'll register an MLflow model, which is stored in the local `model` folder. 

In [4]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

model_name="diabetes-mlflow"
model=ml_client.models.create_or_update(
    Model(
        name=model_name,
        path="./model",
        type=AssetTypes.MLFLOW_MODEL
    )
)

Uploading model (0.0 MBs): 100%|██████████| 2424/2424 [00:00<00:00, 32376.79it/s]




## Create a batch endpoint

A batch endpoint is an HTTPS endpoint that applications can call to trigger a batch scoring job. A batch endpoint name needs to be unique within an Azure region. You'll use the `datetime` function to generate a unique name based on the current date and time. 

In [5]:
import datetime
endpoint_name="batch-"+datetime.datetime.now().strftime("%m%d%H%M%f")
endpoint_name

'batch-08111955302196'

To create an endpoint with the `BatchEndpoint` class, you need to specify the name and optionally a description. After creating an endpoint, you'll deploy a model to the endpoint.

In [6]:
from azure.ai.ml.entities import BatchEndpoint

endpoint=BatchEndpoint(
    name=endpoint_name,
    description="A Batch endpoint for classifying diabetes in patients"
    )

ml_client.batch_endpoints.begin_create_or_update(endpoint=endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created before continuing! A green notification should appear in the studio. </p>

## Create the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. 

Since you're deploying an MLflow model, you don't need a scoring script or define the environment. Azure Machine Learning will automatically create those assets for you. The `MLmodel` file in the `model` folder is used to understand what the expected inputs and outputs are of the model.

You'll deploy a model with the following parameters:

- `name`: Name of the deployment.
- `description`: Optional description to further clarify what the deployment represents.
- `endpoint_name`: Name of the previously created endpoint the model should be deployed to.
- `model`: Name of the registered model.
- `compute`: Compute to be used when invoking the deployed model to generate predictions.
- `instance_count`: Count of compute nodes to use for generating predictions.
- `max_concurrency_per_instance`: Maximum number of parallel scoring script runs per compute node.
- `mini_batch_size`: Number of files passed per scoring script run.
- `output_action`: Each new prediction will be appended as a new row to the output file.
- `output_file_name`: File to which predictions will be appended.
- `retry_settings`: Settings for a mini-batch fails.
- `logging_level`: The log verbosity level. Allowed values are `warning`, `info`, and `debug`. 

Running the following cell will configure and create the deployment.

In [12]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment=BatchDeployment(
    name="classifier-diabetes-mlflow",
    description="A diabetes classifier",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info"
    )
ml_client.batch_deployments.begin_create_or_update(deployment=deployment)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio. </p>

You can deploy multiple models to an endpoint. You can set the default deployment to specify which model should be used by default when calling a batch endpoint.

In [13]:
endpoint.defaults={}
endpoint.defaults['deployment_name']=deployment.name 

ml_client.batch_endpoints.begin_create_or_update(endpoint=endpoint)


<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the default deployment is set before continuing! A green notification should appear in the studio. </p>

## Prepare the data for batch predictions

In the `data` folder you'll find CSV files with unlabeled data. You'll create a data asset that points to the files in the `data` folder, which you'll use as input for the batch job.

In [22]:
# dataset to data asset
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path="./data"
dataset_name="patient-data-unlabelled"

patient_dataset_unlabelled=Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabelled dataset for diabetes classification",
    name=dataset_name
)

ml_client.data.create_or_update(data=patient_dataset_unlabelled)

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'patient-data-unlabelled', 'description': 'An unlabelled dataset for diabetes classification', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/resourceGroups/rg-dp100-l4328d70014b0413aa7/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-l4328d70014b0413aa7/data/patient-data-unlabelled/versions/1', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci4328d70014b0413aa7/code/Users/ritishadhikari/azure-ml-labs/Labs/11', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7ff3726f95b0>, 'serialize': <msrest.serialization.Serializer object at 0x7ff3726f9c70>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/18a1f27f-edf5-495e-

In [30]:
patient_dataset_unlabelled.name, patient_dataset_unlabelled.id

('patient-data-unlabelled',
 '/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/resourceGroups/rg-dp100-l4328d70014b0413aa7/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-l4328d70014b0413aa7/data/patient-data-unlabelled/versions/1')

In [29]:
patient_dataset_unlabelled=ml_client.data.get(
    name="patient-data-unlabelled",
    label="latest"
)

## Submit the job

Now that you have deployed a model to a batch endpoint, and have an unlabeled data asset, you're ready to invoke the endpoint to generate predictions on the unlabeled data.

First, you'll define the input by referring to the registered data asset. Then, you'll invoke the endpoint, which will submit a pipeline job. You can use the job URL to monitor it in the Studio. The job will contain a child job that represents the running of the (generated) scoring script to get the predictions.

In [31]:
# Creating the input as the data asset
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input=Input(
    type=AssetTypes.URI_FOLDER,
    path=patient_dataset_unlabelled.id
)

In [32]:
# Fetching Predictions for the input through the endpoint name
job=ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name,
    input=input
)

ml_client.jobs.get(name=job)

Experiment,Name,Type,Status,Details Page
batch-08111955302196,batchjob-2db2b8c9-4664-46f4-ae8a-4219bbdad05e,pipeline,Preparing,Link to Azure Machine Learning studio


## Get the results

When the pipeline job that invokes the batch endpoint is completed, you can view the results. All predictions are collected in the `predictions.csv` file that is stored in the default datastore. You can download the file and visualize the data by running the following cells. 

In [33]:
# Downloading the predictions.csv locally
ml_client.jobs.download(name=job.name,download_path=".", output_name="score")

In [35]:
# reading the predictions.csv
with open(file="predictions.csv",mode="r") as f:
    data=f.read()

In [41]:
# de-serializing the predictions.csv and getting the score
from ast import literal_eval
import pandas as pd

score=pd.DataFrame(
    data=literal_eval(node_or_string=data.replace("\n",",")),
    columns=["file","prediction"]
)


score

,file,prediction
0,patient-data-0.csv,1
1,patient-data-0.csv,1
2,patient-data-0.csv,1
3,patient-data-0.csv,1
4,patient-data-0.csv,1
...,...,...
395,patient-data-3.csv,1
396,patient-data-3.csv,1
397,patient-data-3.csv,1
398,patient-data-3.csv,1


In [45]:
score[score['prediction']!="1"]

,file,prediction


In [46]:
score['prediction'].value_counts()

1    400
Name: prediction, dtype: int64